In [20]:
import os # To import test image files
import cv2 # To work with opencv images
from PIL import Image # Image submodule to work with pillow images
import pytesseract as pt # pytesseract module

In [21]:
test_img_path = "../test_images/bill"
create_path = lambda f : os.path.join(test_img_path, f)

test_image_files = os.listdir(test_img_path)

for f in test_image_files:
    print(f)

20378050-7626749-image-a-39_1572449889341.jpg
AAA.jpeg
bill01.jpg
bill02.jpg
bill_rotate.jpg
image.png
receipt.jpg


In [22]:
def show_image(img_path, size=(800, 800)):
    image = cv2.imread(img_path)
    image = cv2.resize(image, size)
    
    cv2.imshow("IMAGE", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Use Image Preprocessing

In [24]:
image_path = test_image_files[1] # 2, 3, 12, 1, 13, 15
path = create_path(image_path)

In [25]:
img = cv2.imread(path)

print("Image shape: ", path)

if img is not None:
    # Center the image
    img = cv2.copyMakeBorder(img, 50, 50, 50, 50, cv2.BORDER_CONSTANT, value=[255])

    # Up-sample (optional)
    img = cv2.resize(img, (0, 0), fx=2, fy=2)

    # Convert to gray-scale
    gry = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to create a binary image
    _, binary_img = cv2.threshold(gry, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # OCR
    txt = pt.image_to_string(gry, config="--psm 6")

    print(txt)
else:
    print("Failed to load the image.")

# show_image(path)

Image shape:  ../test_images/bill\AAA.jpeg


%, §
ve NA7~UWISG MOTORS
: 01/11/2019
HONDA CIVIC FK6
Remove & Refitting
** Rs.13,750
Repair
** Rs.11,500
Painting (Booth Painting)
“* Rs.31,000
Parts
|. Front Buffer Rs.100,000/=
2. L/H Front Buffer Retainer Rs.10,000/= |
: 3. L/H Head Lamp Rs.115,000/=
4. L/H Head Lamp Gloss Strip Rs.18,000/=
5. L/H Head Lamp Gloss Strip Mounting Brackets — Rs.15,000/=
6. Shell Rs.80,000/=
7. Bonnet Rs.105,000/=
8. Bonnet Hinges 02 Rs.18,000/=
9. L/H Head Lamp Lower Bracket Rs.15,000/=
10.Bonnet Beading Rs.12,000/=
| 1.R/H Head Lamp Lower Bracket Rs.15,000/=
12.Nose Cut Panel (Plastic) Rs.110,000/= |
13.Scan & Programming Rs.5,000/=
14.A/C Gas Rs.4,500/= |
15.Coolant Rs.1,500/=
16.Body Sealant Rs.2,500/=
17.Double Tape Rs.300/=
18. Buffer Clips Rs.700/=
|
Final Bill Amount Rs.683,750/=
Te 777603000 https://www.facebook.com/pages/VTEC-Motors
_____eee



In [26]:
#identify the date
import re

match=re.findall(r'\d+[/.-]\d+[/.-]\d+', txt)

st=" "
st=st.join(match)
print(st)

01/11/2019


In [27]:
import re

def clean_text(text):
    # Remove links
    text = re.sub(r'http\S+', '', text)
    
    # Remove email addresses
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text)
    
    # Remove phone numbers
    text = re.sub(r'\b\d{10}\b', '', text)
    
    # Remove dates (assuming in dd/mm/yyyy format)
    text = re.sub(r'\b\d{1,2}/\d{1,2}/\d{4}\b', '', text)
    
   # Remove price format like Rs.100,000/=
    text = re.sub(r'Rs\.\d+(,\d+)*\/=', '', text)
    
    
    
    return text.strip()


ocr_text = txt
# Clean the text
cleaned_text = clean_text(ocr_text)

# Split the cleaned text into sentences and add to a list
sentences_list = cleaned_text.split('\n')

# Print the list of sentences
print(cleaned_text)




%, §
ve NA7~UWISG MOTORS
: 
HONDA CIVIC FK6
Remove & Refitting
** Rs.13,750
Repair
** Rs.11,500
Painting (Booth Painting)
“* Rs.31,000
Parts
|. Front Buffer 
2. L/H Front Buffer Retainer  |
: 3. L/H Head Lamp 
4. L/H Head Lamp Gloss Strip 
5. L/H Head Lamp Gloss Strip Mounting Brackets — 
6. Shell 
7. Bonnet 
8. Bonnet Hinges 02 
9. L/H Head Lamp Lower Bracket 
10.Bonnet Beading 
| 1.R/H Head Lamp Lower Bracket 
12.Nose Cut Panel (Plastic)  |
13.Scan & Programming 
14.A/C Gas  |
15.Coolant 
16.Body Sealant 
17.Double Tape 
18. Buffer Clips 
|
Final Bill Amount 
Te 777603000 
_____eee


In [28]:

import string


def preprocess_text(text):
    # Remove punctuation marks
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove escape characters
    text = text.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')

    #remove unnesesarry  spaces 
    text = re.sub(' +', ' ', text)

    # remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Convert to lowercase
    processed_text = text.lower()

    return processed_text

In [29]:
processed_list = []

for item in sentences_list:
    processed_item = preprocess_text(item)
    processed_list.append(processed_item)


In [30]:
processed_list = list(filter(None, processed_list))
processed_list = [item for item in processed_list if item.strip()]
processed_list

[' §',
 've nauwisg motors',
 'honda civic fk',
 'remove refitting',
 ' rs',
 'repair',
 ' rs',
 'painting booth painting',
 '“ rs',
 'parts',
 ' front buffer ',
 ' lh front buffer retainer ',
 '  lh head lamp ',
 ' lh head lamp gloss strip ',
 ' lh head lamp gloss strip mounting brackets — ',
 ' shell ',
 ' bonnet ',
 ' bonnet hinges  ',
 ' lh head lamp lower bracket ',
 'bonnet beading ',
 ' rh head lamp lower bracket ',
 'nose cut panel plastic ',
 'scan programming ',
 'ac gas ',
 'coolant ',
 'body sealant ',
 'double tape ',
 ' buffer clips ',
 'final bill amount ',
 'te  ',
 'eee']

In [31]:
print("Processed Items:")
for item in processed_list:
    print(item)

Processed Items:
 §
ve nauwisg motors
honda civic fk
remove refitting
 rs
repair
 rs
painting booth painting
“ rs
parts
 front buffer 
 lh front buffer retainer 
  lh head lamp 
 lh head lamp gloss strip 
 lh head lamp gloss strip mounting brackets — 
 shell 
 bonnet 
 bonnet hinges  
 lh head lamp lower bracket 
bonnet beading 
 rh head lamp lower bracket 
nose cut panel plastic 
scan programming 
ac gas 
coolant 
body sealant 
double tape 
 buffer clips 
final bill amount 
te  
eee
